In [1]:
# Imports
import os
import pandas as pd


# Clinical data extraction

In [2]:
#Read in the data
data_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/dataset_CAESAR_dec23_MASTER.xlsx"
all_data = pd.read_excel(data_path)

pat_resp_data_path = "C:/Users/P095550/OneDrive - Amsterdam UMC/Documenten/GitHub/CRLM-morph-features/CAIRO5_path_responseTRG__dec2023.xlsx"
pat_resp_data = pd.read_excel(pat_resp_data_path)

### Training input

In [4]:
training_data = all_data[["SubjectKey", "sex", "Age", "largestsize", "distribution", "synchr", "sidedness", "KRAS", "NRAS", "BRAF", "treatment"]]
training_data.head()


,SubjectKey,sex,Age,largestsize,distribution,synchr,sidedness,KRAS,NRAS,BRAF,treatment
0,1,Female,56,62,Bilobar,Synchronous,Left,0,0,1,FOLFOX-B
1,2,Male,65,34,Bilobar,Synchronous,Left,0,0,0,FOLFOX-P
2,3,Male,55,48,Bilobar,Synchronous,Left,1,0,0,FOLFOX-B
3,4,Female,35,67,Bilobar,Synchronous,Left,0,0,0,FOLFOX-P
4,5,Female,57,38,Bilobar,Synchronous,Right,0,0,0,FOLFIRI-B


In [ ]:
# Correctly apply transformations to columns
training_data["male"] = training_data["sex"].apply(lambda x: 1 if x == "Male" else 0)
training_data["synchr"] = training_data["synchr"].apply(lambda x: 1 if x == "Synchronous" else 0)
training_data["left_sidedness"] = training_data["sidedness"].apply(lambda x: 1 if x == "Left" else 0)
training_data["bevacizumab"] = training_data["treatment"].apply(lambda x: 1 if x.endswith("B") else 0)
training_data["bilobar"] = training_data["distribution"].apply(lambda x: 1 if x == "Bilobar" else 0)

# Drop the original columns if they are no longer needed
training_data.drop(columns=["sex", "sidedness", "treatment", "distribution"], inplace=True)

In [6]:
nan_counts = training_data.isna().sum()
print(nan_counts)

SubjectKey        0
Age               0
largestsize       0
synchr            0
KRAS              0
NRAS              0
BRAF              0
male              0
left_sidedness    0
bevacizumab       0
bilobar           0
dtype: int64


In [ ]:
# Subset the data to only include patients which we have CT scans for

paired_scans_path = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_scans"
subject_keys = set()
for filename in os.listdir(paired_scans_path):
    subject_keys.add(int(filename[6:9]))

training_data = training_data[training_data["SubjectKey"].isin(subject_keys)]

training_data.drop(columns=["SubjectKey"], inplace=True)

In [14]:

# Save the dataframe as a CSV file
training_data.to_csv("training_data/training_data.csv", index=False)

### Training labels